In [1]:
import numpy as np
import tensorflow as tf

import autokeras as ak

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd

2022-01-29 19:37:58.726832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64
2022-01-29 19:37:58.726859: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
covid_path = 'dataset/COVID-19_Radiography_Dataset/COVID'
noncovid_path = 'dataset/COVID-19_Radiography_Dataset/Normal'

covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

covid_files = covid_files[:500]
noncovid_files = noncovid_files[:500]

In [3]:
# check dimensions
import cv2

covid_images = []
covid_labels = []

noncovid_images = []
noncovid_labels = []

for i in range(len(covid_files)):
  image = cv2.imread(covid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(299,299))
  covid_images.append(image)
  covid_labels.append('Chest_COVID')
  
for i in range(len(noncovid_files)):
  image = cv2.imread(noncovid_files[i])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(299,299))
  noncovid_images.append(image)
  noncovid_labels.append('Chest_NonCOVID')

In [4]:
covid_images = np.array(covid_images) / 255
noncovid_images = np.array(noncovid_images) / 255

In [5]:
# split into training and testing
covid_x_train, covid_x_test, covid_y_train, covid_y_test = train_test_split(covid_images, covid_labels, test_size=0.2)
noncovid_x_train, noncovid_x_test, noncovid_y_train, noncovid_y_test = train_test_split(noncovid_images, noncovid_labels, test_size=0.2)


X_train = np.concatenate((noncovid_x_train, covid_x_train), axis=0)
X_test = np.concatenate((noncovid_x_test, covid_x_test), axis=0)
y_train = np.concatenate((noncovid_y_train, covid_y_train), axis=0)
y_test = np.concatenate((noncovid_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
#y_train = to_categorical(y_train)

y_test = LabelBinarizer().fit_transform(y_test)
#y_test = to_categorical(y_test)

In [6]:
X_train.shape, X_test.shape, y_train.shape

((800, 299, 299, 3), (200, 299, 299, 3), (800, 1))

In [7]:
clf = ak.ImageClassifier(overwrite=True, max_trials=1)


In [8]:
clf.fit(X_train, y_train, epochs=50)

Trial 1 Complete [00h 13m 24s]
val_loss: 3.661316938519121e-08

Best val_loss So Far: 3.661316938519121e-08
Total elapsed time: 00h 13m 24s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
25/25 [==============================] - 78s 3s/step - loss: 62.4533 - accuracy: 0.7088
Epoch 2/50
25/25 [==============================] - 76s 3s/step - loss: 3.7225 - accuracy: 0.2087
Epoch 3/50
25/25 [==============================] - 76s 3s/step - loss: 0.7049 - accuracy: 0.2438
Epoch 4/50
25/25 [==============================] - 76s 3s/step - loss: 0.6985 - accuracy: 0.4313
Epoch 5/50
25/25 [==============================] - 76s 3s/step - loss: 0.6969 - accuracy: 0.2962
Epoch 6/50
25/25 [==============================] - 76s 3s/step - loss: 0.6938 - accuracy: 0.5763
Epoch 7/50
25/25 [==============================] - 76s 3s/step - loss: 0.6803 - accuracy: 0.5250
Epoch 8/50
25/25 [==============================] - 76s 3s/step - loss: 0.7563 - accuracy: 0.4750
Epoch 9/50
25/25 [===================

2022-01-29 21:02:40.952387: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [9]:
model = clf.export_model()

In [10]:
type(model)

tensorflow.python.keras.engine.functional.Functional

In [11]:
model.save('covid_classifier', save_format='tf')

INFO:tensorflow:Assets written to: covid_classifier/assets


In [12]:
predicted_y = clf.predict(X_test)

7/7 [==============================] - 5s 571ms/step


In [15]:
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.79      0.82      0.80       100
           1       0.81      0.78      0.80       100

    accuracy                           0.80       200
   macro avg       0.80      0.80      0.80       200
weighted avg       0.80      0.80      0.80       200



In [18]:
print(clf.evaluate(X_test, y_test))

7/7 [==============================] - 5s 492ms/step - loss: 0.9187 - accuracy: 0.8000
[0.9186539649963379, 0.800000011920929]


In [19]:
X_test.shape

(200, 299, 299, 3)

In [20]:
X_test[1].shape

(299, 299, 3)